# Libraries

In [1]:
__author__ = "Gabriele Greco"
import random
import logging
from collections import namedtuple
from collections import Counter
from matplotlib import pyplot as plt

# Functions

In [2]:
Individual = namedtuple("Individual", ["genome", "fitness"])

def evaluate(gen):
    count = Counter()
    count.update(sum((e for e in gen), start=()))
    return len(count), -count.total() # len(count) length of my gen elements (without duplicate)

def tournament(population, tournament_size=2): # Return from 2 individuals the one with higher fitness
    return max(random.choices(population, k=tournament_size), key=lambda i: i.fitness)

def cross_over(g1, g2):
    if (len(g1) > len(g2)):
        len_small = len(g2)
    else:
        len_small = len(g1)
    cut2 = random.randint(1, len_small)
    cut1 = random.randint(cut2-1, len_small-1)

    return set(list(g1)[cut1:] + list(g2)[:cut2])

def mutation(g, all_lists):
    new_gen = set(g) 
    r = random.choice(list(new_gen)) # modify our solution randomly
    new_gen.remove(r)  # remove one random element from our solution
    new_set = random.choice(list(all_lists - g)) # we pick a new random element given by our full list of sets minus the elements that we already have
    new_gen.add(tuple(new_set)) # add the new element to our solution
    return new_gen

# Initial Population

In [3]:
def problem(N, seed=None): # Generator of sets
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

def create_population(all_lists):
    population = list()
    for genome in all_lists:
        gen = set()
        gen.add(genome)
        population.append(Individual(gen, evaluate(gen)))
    return population

# Evolution

In [4]:
def evolution(population, population_size, mutation_rate, all_lists, NUM_GENERATIONS):
    offspring_size = int(population_size / 2)    
    fitness_log = [(0, i.fitness) for i in population]
    for g in range(NUM_GENERATIONS):
        offspring = list()
        for i in range(offspring_size):
            if random.random() < mutation_rate: # mutation on one random gene
                parent = tournament(population)
                o = mutation(parent.genome, all_lists)
            else: # otherwise we pick 2 random parents and thanks to the crossover between them we create a new individuals
                parent1 = tournament(population)  # parent1
                parent2 = tournament(population)  # parent2
                o = cross_over(parent1.genome, parent2.genome)
            f = evaluate(o) # fitness function
            fitness_log.append((g + 1, f))
            offspring.append(Individual(o, f)) # all offspring created in one generation
        population += offspring # after one generation add the offspring to the population
        population = sorted(population, key=lambda i: i.fitness, reverse=True)[:population_size]  # we pick only the best population_size individuals
    return population, fitness_log

# Genetic Algorithm

In [5]:
def main():
    logging.getLogger().setLevel(logging.INFO)
    for N in [5, 10, 20, 50, 100, 500, 1000]: # 5, 10, 20, 50, 100, 500, 1000 
        NUM_GENERATIONS = (N * 2) + 50
        mutation_rate = 0.55 # started from 0.3 -> 0.45 -> 0.55 -> 0.90, 0.55 is the best
        all_lists = set(tuple(_) for _ in problem(N, 42)) # SEED = 42
        population = create_population(all_lists)
        population_size = len(population)

        solution, fitness_log = evolution(population, population_size, mutation_rate, all_lists, NUM_GENERATIONS)
        w = (-solution[0][1][1])
        logging.info(f" Solution for N={N:,}: " + f"w={w} " + f"(bloat={(w-N)/N*100:.0f}%) " + f"Fitness calls={len(fitness_log)}")

In [6]:
if __name__ == '__main__':
    main()

INFO:root: Solution for N=5: w=5 (bloat=0%) Fitness calls=310
INFO:root: Solution for N=10: w=10 (bloat=0%) Fitness calls=1512
INFO:root: Solution for N=20: w=24 (bloat=20%) Fitness calls=1564
INFO:root: Solution for N=50: w=79 (bloat=58%) Fitness calls=16113
INFO:root: Solution for N=100: w=187 (bloat=87%) Fitness calls=53677
INFO:root: Solution for N=500: w=1338 (bloat=168%) Fitness calls=951009
INFO:root: Solution for N=1,000: w=3024 (bloat=202%) Fitness calls=3712069
